この問題をbashで解決する方法を、詳細な図解とともに説明します。

In [ ]:
#!/bin/bash

# file.txtの内容を転置する

# 方法1: awkを使用したシンプルな解法
# Analyze Complexity
# Runtime 72 ms
# Beats 29.70%
# Memory 7.82 MB
# Beats 13.13%
awk '
{
    # 各フィールドを配列に格納
    for (i = 1; i <= NF; i++) {
        a[NR, i] = $i
    }
}
NF > p { p = NF }  # 最大列数を記録
END {
    # 転置して出力
    for (j = 1; j <= p; j++) {
        str = a[1, j]
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]
        }
        print str
    }
}
' file.txt

## 問題の理解

この問題は**行列の転置**と同じ操作です。元のファイルの行と列を入れ替えます。

### 入力例の視覚化

```
元のファイル (2列 × 3行):
┌─────────────┐
│ name   age  │  ← 行1
│ alice  21   │  ← 行2
│ ryan   30   │  ← 行3
└─────────────┘
   ↑      ↑
  列1    列2
```

### 出力の視覚化

```
転置後 (3列 × 2行):
┌────────────────────┐
│ name  alice  ryan  │  ← 元の列1が行1に
│ age   21     30    │  ← 元の列2が行2に
└────────────────────┘
```

## 解法の詳細説明

### **ステップ1: データの読み込みと配列への格納**

```
awk '{
    for (i = 1; i <= NF; i++) {
        a[NR, i] = $i
    }
}'
```

**図解:**

```
読み込み処理:

NR=1: name age
      ↓    ↓
   a[1,1] a[1,2]
    name   age

NR=2: alice 21
      ↓     ↓
   a[2,1] a[2,2]
   alice   21

NR=3: ryan 30
      ↓    ↓
   a[3,1] a[3,2]
   ryan    30

結果の2次元配列:
      列1    列2
    ┌──────┬─────┐
行1 │ name │ age │
    ├──────┼─────┤
行2 │alice │ 21  │
    ├──────┼─────┤
行3 │ ryan │ 30  │
    └──────┴─────┘
```

**変数の説明:**
- `NR`: 現在の行番号 (Number of Records)
- `NF`: 現在の行のフィールド数 (Number of Fields)
- `$i`: i番目のフィールド
- `a[NR, i]`: 2次元配列 (行, 列)

### **ステップ2: 最大列数の記録**

```
NF > p { p = NF }
```

各行のフィールド数をチェックし、最大値を`p`に保存します。

### **ステップ3: 転置して出力**

```
END {
    for (j = 1; j <= p; j++) {
        str = a[1, j]
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]
        }
        print str
    }
}
```

**図解:**

```
転置処理:

j=1のとき (元の列1):
┌─────┐
│name │ a[1,1]  →  str = "name"
├─────┤
│alice│ a[2,1]  →  str = "name alice"
├─────┤
│ryan │ a[3,1]  →  str = "name alice ryan"
└─────┘
出力: "name alice ryan"

j=2のとき (元の列2):
┌─────┐
│age  │ a[1,2]  →  str = "age"
├─────┤
│21   │ a[2,2]  →  str = "age 21"
├─────┤
│30   │ a[3,2]  →  str = "age 21 30"
└─────┘
出力: "age 21 30"
```

## 別解: より簡潔なワンライナー

```bash
awk '{for(i=1;i<=NF;i++)a[i]=a[i]" "$i}END{for(i=1;i<=NF;i++)print substr(a[i],2)}' file.txt
```

この方法では、各列のデータを文字列として連結していきます。

## 実行方法

```bash
# スクリプトを実行権限付きで保存
chmod +x transpose.sh

# 実行
./transpose.sh

# または直接awkコマンドを実行
awk '{for(i=1;i<=NF;i++)a[NR,i]=$i}NF>p{p=NF}END{for(j=1;j<=p;j++){str=a[1,j];for(i=2;i<=NR;i++)str=str" "a[i,j];print str}}' file.txt
```

## まとめ

この解法のポイント:
1. **2次元配列**を使ってデータを格納
2. **外側ループ**で列を、**内側ループ**で行を処理
3. 行と列のインデックスを入れ替えることで転置を実現

この手法は任意のサイズの行列に対応できます!

現在の解法は機能的には正しいですが、パフォーマンスとメモリ使用量に改善の余地があります。より効率的な解法を提案します。

In [ ]:
#!/bin/bash

# 最適化版: 文字列連結を使用してメモリ効率を改善
# Analyze Complexity
# Runtime 59 ms
# Beats 93.40%
# Memory 3.99 MB
# Beats 91.05%

awk '
{
    for (i = 1; i <= NF; i++) {
        if (NR == 1) {
            # 最初の行: 配列を初期化
            a[i] = $i
        } else {
            # 2行目以降: スペース区切りで連結
            a[i] = a[i] " " $i
        }
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]
    }
}
' file.txt

## パフォーマンス比較と改善点

### **元の解法の問題点**

```
問題1: 2次元配列の使用
┌─────────────────────┐
│ a[1,1] a[1,2]       │  メモリ使用量: O(行数 × 列数)
│ a[2,1] a[2,2]       │  各セルを個別に保存
│ a[3,1] a[3,2]       │  アクセスオーバーヘッドが大きい
└─────────────────────┘

問題2: 二重ループでの文字列連結
END {
    for (j = 1; j <= p; j++) {
        str = a[1, j]              ← 毎回新しい文字列を作成
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]  ← 文字列を何度も再構築
        }
        print str
    }
}
```

### **改善版の利点**

```
改善1: 1次元配列 + 文字列連結
┌──────────────────────┐
│ a[1] = "name alice ryan" │  メモリ: O(列数)のみ
│ a[2] = "age 21 30"       │  各列を1つの文字列として保存
└──────────────────────────┘

改善2: 処理中に連結
{
    for (i = 1; i <= NF; i++) {
        a[i] = a[i] " " $i  ← 読み込みながら連結
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]           ← 単純な出力のみ
    }
}
```

## さらなる最適化オプション

### **オプション1: rs コマンド (最速)**

```bash
# rsコマンドを使用 (利用可能な場合)
rs -T < file.txt
```

**特徴:**
- C言語で実装された専用ツール
- 最も高速
- ただし、一部のシステムでは利用不可

### **オプション2: 改良版awkスクリプト**

In [ ]:
#!/bin/bash

# 方法1: 最適化版 (推奨) - メモリ効率が良い
# Analyze Complexity
# Runtime 63 ms
# Beats 77.60%
# Memory 3.99 MB
# Beats 91.05%

awk '{for(i=1;i<=NF;i++)a[i]=a[i](NR>1?" ":"")$i}END{for(i=1;i<=NF;i++)print a[i]}' file.txt

# 方法2: 可読性重視版
# Analyze Complexity
# Runtime 67 ms
# Beats 53.43%
# Memory 3.98 MB
# Beats 91.05%

awk '
{
    for (i = 1; i <= NF; i++) {
        a[i] = a[i] (NR > 1 ? " " : "") $i
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]
    }
}
' file.txt

# 方法3: pasteコマンド利用 (Bash純正アプローチ)
# Analyze Complexity
# Runtime 269 ms
# Beats 5.64%
# Memory 3.59 MB
# Beats 100.00%

cols=$(head -1 file.txt | wc -w)
for i in $(seq 1 $cols); do
    cut -d' ' -f$i file.txt | paste -sd' '
done

## 問題の理解

この問題は**行列の転置**と同じ操作です。元のファイルの行と列を入れ替えます。

### 入力例の視覚化

```mermaid
graph TD
    subgraph "元のファイル (2列 × 3行)"
        A["行1: name   age"]
        B["行2: alice  21"]
        C["行3: ryan   30"]
    end
    
    subgraph "列の構造"
        D["列1: name, alice, ryan"]
        E["列2: age, 21, 30"]
    end
    
    A --> D
    A --> E
    B --> D
    B --> E
    C --> D
    C --> E
    
    style A fill:#e1f5ff
    style B fill:#e1f5ff
    style C fill:#e1f5ff
    style D fill:#fff4e1
    style E fill:#fff4e1
```

### 出力の視覚化

```mermaid
graph LR
    subgraph "転置後 (3列 × 2行)"
        A["行1: name  alice  ryan"]
        B["行2: age   21     30"]
    end
    
    subgraph "元の列が行に変換"
        C["元の列1 → 行1"]
        D["元の列2 → 行2"]
    end
    
    C --> A
    D --> B
    
    style A fill:#d1fae5
    style B fill:#d1fae5
    style C fill:#fef3c7
    style D fill:#fef3c7
```

## 解法の詳細説明

### **ステップ1: データの読み込みと配列への格納**

```bash
awk '{
    for (i = 1; i <= NF; i++) {
        a[NR, i] = $i
    }
}'
```

**図解:**

```mermaid
flowchart TD
    subgraph "読み込み処理"
        A["NR=1: name age"] --> B["a[1,1]=name<br/>a[1,2]=age"]
        C["NR=2: alice 21"] --> D["a[2,1]=alice<br/>a[2,2]=21"]
        E["NR=3: ryan 30"] --> F["a[3,1]=ryan<br/>a[3,2]=30"]
    end
    
    subgraph "結果の2次元配列"
        G["行1: [name, age]"]
        H["行2: [alice, 21]"]
        I["行3: [ryan, 30]"]
    end
    
    B --> G
    D --> H
    F --> I
    
    style A fill:#e1f5ff
    style C fill:#e1f5ff
    style E fill:#e1f5ff
    style B fill:#fff4e1
    style D fill:#fff4e1
    style F fill:#fff4e1
    style G fill:#d1fae5
    style H fill:#d1fae5
    style I fill:#d1fae5
```

**変数の説明:**
- `NR`: 現在の行番号 (Number of Records)
- `NF`: 現在の行のフィールド数 (Number of Fields)
- `$i`: i番目のフィールド
- `a[NR, i]`: 2次元配列 (行, 列)

### **ステップ2: 最大列数の記録**

```bash
NF > p { p = NF }
```

各行のフィールド数をチェックし、最大値を`p`に保存します。

### **ステップ3: 転置して出力**

```bash
END {
    for (j = 1; j <= p; j++) {
        str = a[1, j]
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]
        }
        print str
    }
}
```

**図解:**

```mermaid
flowchart TD
    subgraph "j=1 (元の列1)"
        A["a[1,1]=name"] --> B["str='name'"]
        C["a[2,1]=alice"] --> D["str='name alice'"]
        E["a[3,1]=ryan"] --> F["str='name alice ryan'"]
        F --> G["出力: 'name alice ryan'"]
    end
    
    subgraph "j=2 (元の列2)"
        H["a[1,2]=age"] --> I["str='age'"]
        J["a[2,2]=21"] --> K["str='age 21'"]
        L["a[3,2]=30"] --> M["str='age 21 30'"]
        M --> N["出力: 'age 21 30'"]
    end
    
    style A fill:#e1f5ff
    style C fill:#e1f5ff
    style E fill:#e1f5ff
    style H fill:#e1f5ff
    style J fill:#e1f5ff
    style L fill:#e1f5ff
    style G fill:#d1fae5
    style N fill:#d1fae5
```

## パフォーマンス比較と改善点

### **元の解法の問題点**

```mermaid
graph TD
    subgraph "問題1: 2次元配列の使用"
        A["メモリ使用量: O(行数 × 列数)"]
        B["各セルを個別に保存"]
        C["アクセスオーバーヘッドが大きい"]
    end
    
    subgraph "問題2: 二重ループでの文字列連結"
        D["毎回新しい文字列を作成"]
        E["文字列を何度も再構築"]
        F["パフォーマンス低下"]
    end
    
    A --> D
    B --> E
    C --> F
    
    style A fill:#fee2e2
    style B fill:#fee2e2
    style C fill:#fee2e2
    style D fill:#fef3c7
    style E fill:#fef3c7
    style F fill:#fef3c7
```

### **改善版の利点**

```mermaid
graph LR
    subgraph "改善1: 1次元配列 + 文字列連結"
        A["メモリ: O(列数)のみ"]
        B["各列を1つの文字列として保存"]
        C["a[1]='name alice ryan'<br/>a[2]='age 21 30'"]
    end
    
    subgraph "改善2: 処理中に連結"
        D["読み込みながら連結"]
        E["単純な出力のみ"]
        F["パフォーマンス向上"]
    end
    
    A --> D
    B --> E
    C --> F
    
    style A fill:#d1fae5
    style B fill:#d1fae5
    style C fill:#d1fae5
    style D fill:#dbeafe
    style E fill:#dbeafe
    style F fill:#dbeafe
```

## パフォーマンス分析

### **時間計算量の比較**

```mermaid
graph TD
    subgraph "元の解法"
        A["読み込み: O(行数 × 列数)"]
        B["出力: O(行数 × 列数)"]
        C["合計: O(2 × 行数 × 列数)"]
    end
    
    subgraph "改善版"
        D["読み込み+連結: O(行数 × 列数)"]
        E["出力: O(列数)"]
        F["合計: O(行数 × 列数 + 列数)"]
    end
    
    A --> B --> C
    D --> E --> F
    
    C -.->|"遅い"| G[パフォーマンス比較]
    F -.->|"速い"| G
    
    style C fill:#fee2e2
    style F fill:#d1fae5
    style G fill:#e0e7ff
```

### **メモリ使用量の比較**

```mermaid
graph TD
    subgraph "元の解法"
        A["2次元配列:<br/>行数 × 列数 個の要素"]
        B["一時文字列:<br/>列数 個"]
        C["合計: O(行数 × 列数)"]
    end
    
    subgraph "改善版"
        D["1次元配列:<br/>列数 個の文字列"]
        E["各文字列長:<br/>行数 × 平均単語長"]
        F["合計: O(列数 × 行数)<br/>だが実装が効率的"]
    end
    
    A --> B --> C
    D --> E --> F
    
    C -.->|"メモリ多"| G[メモリ比較]
    F -.->|"メモリ少"| G
    
    style C fill:#fee2e2
    style F fill:#d1fae5
    style G fill:#e0e7ff
```

### **実行例の詳細図解**

```mermaid
sequenceDiagram
    participant Input as file.txt
    participant AWK as AWK処理
    participant Array as 配列a[]
    participant Output as 出力
    
    Note over Input: name age<br/>alice 21<br/>ryan 30
    
    Input->>AWK: NR=1: name age
    AWK->>Array: a[1]="name"<br/>a[2]="age"
    
    Input->>AWK: NR=2: alice 21
    AWK->>Array: a[1]="name alice"<br/>a[2]="age 21"
    
    Input->>AWK: NR=3: ryan 30
    AWK->>Array: a[1]="name alice ryan"<br/>a[2]="age 21 30"
    
    Array->>Output: print a[1]
    Note over Output: name alice ryan
    
    Array->>Output: print a[2]
    Note over Output: age 21 30
```

## ベンチマーク予想

```mermaid
graph LR
    subgraph "改善版の期待値"
        A["Runtime: 40-50ms<br/>(約40-50%改善)"]
        B["Memory: 4-5MB<br/>(約35-40%改善)"]
    end
    
    subgraph "主な改善要因"
        C["配列アクセスの削減:<br/>2次元→1次元"]
        D["ループネストの削減:<br/>二重ループ→単一ループ"]
        E["文字列操作の最適化:<br/>逐次連結→直接連結"]
        F["条件分岐の最適化:<br/>三項演算子の使用"]
    end
    
    C --> A
    D --> A
    E --> B
    F --> B
    
    style A fill:#d1fae5
    style B fill:#d1fae5
    style C fill:#dbeafe
    style D fill:#dbeafe
    style E fill:#dbeafe
    style F fill:#dbeafe
```

この最適化により、上位50-70%のパフォーマンスが期待できます!

主な変更点：
1. ASCII図をmermaid形式のグラフに変換
2. フローチャート、シーケンス図、グラフを適切に使用
3. 色分けでわかりやすく視覚化
4. 矢印や関係性を明確に表現